In [1]:
import sys
sys.path.append('../')
import torch
import matplotlib.pyplot as plt
from notebook_setup import device
from ocd.models.oslow import OSlow
from ocd.data.synthetic.graph_generator import GraphGenerator
from ocd.data.synthetic.utils import RandomGenerator
from ocd.data.synthetic.parametric import AffineParametericDataset
from ocd.data.synthetic.nonparametric import AffineNonParametericDataset
from ocd.models.normalization import ActNorm
from ocd.models.postnonlinear import InPlaceTransform 
from ocd.models.masked import MaskedAffineFlowTransform

%load_ext autoreload
%autoreload 2
print(device)

cuda


Perform a derivative test to check if the dependencies of the flow transforms are correct.

In [2]:
# create a 128 by 3 tensor that is random
x = torch.randn((128, 3)).to(device)

transform_inplace = InPlaceTransform(
    (3,),
    num_bins=10,
    tail_bound=1.0,
    identity_init=False,
).to(device)
transform_masked = MaskedAffineFlowTransform(
    # architecture
    in_features= 3,
    layers = [10, 10, 10],
    dropout = None,
    residual = False,
    activation = torch.nn.ReLU(),
    # additional flow args
    additive = False,
    normalization = ActNorm,
    # ordering
    ordering = torch.IntTensor([1, 0, 2]),
).to(device)
transform_inplace.eval()
transform_masked.eval()

MaskedAffineFlowTransform(
  (masked_mlp_shift): MaskedMLP(
    (0): MaskedBlock(
      in_features=3, out_features=30, bias=True
      (activation): ReLU()
    )
    (1-2): 2 x MaskedBlock(
      in_features=30, out_features=30, bias=True
      (activation): ReLU()
    )
    (3): MaskedBlock(in_features=30, out_features=3, bias=True)
  )
  (masked_mlp_scale): MaskedMLP(
    (0): MaskedBlock(
      in_features=3, out_features=30, bias=True
      (activation): ReLU()
    )
    (1-2): 2 x MaskedBlock(
      in_features=30, out_features=30, bias=True
      (activation): ReLU()
    )
    (3): MaskedBlock(
      in_features=30, out_features=3, bias=True
      (activation): ReLU()
    )
    (4): MaskedBlock(in_features=3, out_features=3, bias=True)
  )
  (normalization): ActNorm()
)

In [3]:


ft_jacobian = torch.func.jacrev(transform_inplace)(torch.zeros_like(x))
print(ft_jacobian[0][0, :, 0, :].cpu().detach().numpy())

def func(x):
    return transform_masked(x, perm_mat=None)

ft_jacobian = torch.func.jacrev(func)(torch.zeros_like(x))
print(ft_jacobian[0][0, :, 0, :].cpu().detach().numpy())


[[1.16621   0.        0.       ]
 [0.        0.9870346 0.       ]
 [0.        0.        1.6504365]]
[[ 1.2771386e+00  1.9475498e-05  0.0000000e+00]
 [ 0.0000000e+00  7.8886610e-01  0.0000000e+00]
 [ 1.7929706e-03 -9.1501847e-03  1.5317332e+00]]
